# Design analysis and combine data from many flights

## Import modules to retrieve data instances and custom organizational structure

### Persona: Analyst

In [ ]:
import metaflow
import opal.flow

s3 = opal.flow.minio_s3fs()
translate_flow = metaflow.Flow("TipTranslateFlow")
tip_catalog_df = metaflow.Flow("TipMidnightCatalog").latest_successful_run.data.catalog_df
tip_catalog_df

### Define analytics and plotting functions

Analytics:

- Join engine turbine speed ARINC429 word table with navigation data from 1553 message table
- Define cruise altitude as the value during periods in which there is little change in the mean
- Return a new table with turbine speed and cruise altitude

Plot:

- Sanity check
- Cruise Altitude as a function of general Altitude 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update({"font.size":18})

altitude = "NAV-25"
altitude_valid = "NAV-0111"
rpm = "N1_RPM_ACTUAL"
rpm_dataset = "Engine_Fan_RPM_N1_ACTUAL_40"

def get_rpm_at_cruise_altitude(t_1553, t_429):
    # get the 1553 and ARINC429 data
    df_1553 = pd.read_parquet(t_1553, filesystem=s3)
    df_429 = pd.read_parquet(t_429, filesystem=s3)
    
    # filter 1553 by the 'altitude valid' bit
    df_1553 = df_1553[df_1553[altitude_valid]]
    
    # join 1553 and ARINC429 data by time
    joined = pd.concat([ df_1553[["time", altitude]], df_429[["time", rpm]] ]).sort_values("time")
    joined[rpm] = joined[rpm].fillna(method="ffill")
    joined = joined.dropna().copy()
    
    # calculate change in altitude over time (time is in nanoseconds)
    joined.loc[:,"diff_altitude"] = joined[altitude].diff() / (joined["time"].diff() / 10**9)
    # smooth out the altitude derivative
    joined.loc[:,"diff_altitude"] = joined["diff_altitude"].rolling(int(1 / 0.04)).mean()
    
    # select only where difference in altitude is small enough
    # and the aircraft is sufficiently above what we think is ground level
    joined = joined[abs(joined["diff_altitude"]) < 0.25]
    start_altitude = joined[altitude].iloc[:10].mean()
    end_altitude = joined[altitude].iloc[-10:].mean()
    min_altitude = max(start_altitude, end_altitude)
    joined = joined[joined[altitude] > min_altitude * 2]
    joined = joined.dropna()
    
    # group periods of time when it was cruising, take the average
    joined['group'] = (joined['time'].diff() > 60 * 10**9).cumsum()
    grouped = joined.groupby('group').mean()
    return grouped[[altitude, rpm]]

def plot_cruise_altitude(t_1553, t_429):
    # get the 1553 and ARINC429 data
    df_1553 = pd.read_parquet(t_1553, filesystem=s3)
    df_429 = pd.read_parquet(t_429, filesystem=s3)
    
    # filter 1553 by the 'altitude valid' big
    df_1553 = df_1553[df_1553[altitude_valid]]
    
    # join 1553 and ARINC429 data by time
    joined = pd.concat([ df_1553[["time", altitude]], df_429[["time", rpm]] ]).sort_values("time")
    joined[rpm] = joined[rpm].fillna(method="ffill")
    joined = joined.dropna().copy()
    
    # calculate change in altitude over time (time is in nanoseconds)
    joined.loc[:,"diff_altitude"] = joined[altitude].diff() / (joined["time"].diff() / 10**9)
    # smooth out the altitude derivative
    joined.loc[:,"diff_altitude"] = joined["diff_altitude"].rolling(int(1 / 0.04)).mean()
    
    # select only where difference in altitude is small enough
    # and the aircraft is sufficiently above what we think is ground level
    joined = joined[abs(joined["diff_altitude"]) < 0.25]
    start_altitude = joined[altitude].iloc[:10].mean()
    end_altitude = joined[altitude].iloc[-10:].mean()
    min_altitude = max(start_altitude, end_altitude)
    joined = joined[joined[altitude] > min_altitude * 2]
    joined = joined.dropna()
    
    ax = joined.plot(kind="scatter", x="time", y=altitude, c="tab:orange", label="Cruise Altitude [ft]", figsize=(13,10))
    df_1553.plot(x="time", y=altitude, ax=ax, label="General Altitude [ft]")
    ax.set_ylabel("Altitude [ft]")
    ax.set_xlabel("Time [Epoch, ns]")

first_row = tip_catalog_df.iloc[0]
print(f"Using chapter 10 {first_row.name}")
plot_cruise_altitude(first_row["NAV"], first_row[rpm_dataset])

### Calculate dataframes of RPM and cruise altitude for _all_ translated data sets, plot

In [ ]:
%%time

from dask.distributed import Client, progress
import dask.bag
import dask.dataframe
import os

with Client(n_workers=16, processes=True) as client:
    dashboard_port = client.cluster.dashboard_link.split(':')[-1].split('/')[0]
    dashboard_link = f"https://opal.metrostar.cloud{os.environ['JUPYTERHUB_SERVICE_PREFIX']}proxy/{dashboard_port}/status"
    print(dashboard_link)
    
    ds_bag = dask.bag.from_sequence(tip_catalog_df.iloc)
    ds_bag = ds_bag.filter(lambda row: row["NAV"] is not None and row[rpm_dataset] is not None)
    ds_bag = ds_bag.map(lambda row: get_rpm_at_cruise_altitude(row["NAV"], row[rpm_dataset]))
    future = client.compute(ds_bag)
    progress(future, notebook=False)
    df = pd.concat(future.result())
   
    df.plot(
        kind="scatter", x=altitude, y=rpm, 
        xlabel="Cruise Altitude [ft]", ylabel="Engine Turbine Speed [RPM, percent max.]",
        title="Turbine Speed at Cruise Altitude (all flights)",
        figsize=(15, 10)
    )

### Calculate a local weighted mean at every point of the complete dataframe

In [ ]:
scale = 1/1500
statistics = np.zeros((len(df), 4))
for i, row in enumerate(df.iloc):
    alt, rpm_ = row
    alt_min = alt - (1 / scale)
    alt_max = alt + (1 / scale)
    
    mean_with = df[(df[altitude] > alt_min) & (df[altitude] < alt_max)].copy()
    mean_with['diff'] = abs(mean_with[altitude] - alt)
    mean_with['scale'] = 1 - mean_with['diff'] * scale
    
    mean = sum(mean_with[rpm] * mean_with['scale']) / sum(mean_with['scale'])
    low = mean_with[rpm].quantile(0.25)
    high = mean_with[rpm].quantile(0.75)
    statistics[i,:] = [alt, mean, low, high]
    
stats_df = pd.DataFrame(statistics, columns=[altitude, "mean", "low", "high"])

ax = df.plot(
    kind="scatter", x=altitude, y=rpm, 
    xlabel="Cruise Altitude [ft]", ylabel="Engine Turbine Speed [RPM, percent max.]",
    title="Turbine Speed at Cruise Altitude (all flights) With Moving Average",
    figsize=(15, 10), legend=True
)
stats_df.plot(
    kind="scatter", x=altitude , y="mean",
    ax=ax, c="black", legend=True
)